# AOC 2025 Day 10

In [1]:
@file:DependsOn("com.toldoven.aoc:aoc-kotlin-notebook:1.1.2")

In [3]:
import com.toldoven.aoc.notebook.AocClient

val aocClient = AocClient.fromEnv().interactiveDay(2025, 10)
val input = aocClient.input()
aocClient.viewPartOne()

java.lang.Exception: This day is not unlocked yet. Unlocks in 17 hours